In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 1.0.0 cuda: True


In [2]:
%run PARAMETERS.py
NAME = 'APG'
MCMC_STEPS = 5
PATH = NAME + '-bmnist-%dsamples' % (SAMPLE_SIZE)
CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda:1')

In [3]:
from model_operations import Init_models, Save_models
models, optimizer = Init_models(FRAME_PIXELS=FRAME_PIXELS, 
                                DIGIT_PIXELS=DIGIT_PIXELS, 
                                HIDDEN_LIST=HIDDEN_LIST, 
                                CUDA=CUDA, 
                                device=DEVICE, 
                                lr=LEARNING_RATE, 
                                RESTORE=False, 
                                PATH=None)

(enc_coor, enc_digit, dec_digit) = models

from crop import *
crop = Crop(digit_size=28, 
            frame_size=64, 
            CUDA=CUDA, 
            device=DEVICE)

In [ ]:
from apg import APG
train(models=models, 
      objective=APG, 
      optimizer=optimizer, 
      data=data, 
      tjs=tjs,
      mcmc_steps=MCMC_STEPS,
      mnist_mean=mnist_mean,
      crop=crop,
      Train_Params=Train_Params,
      CUDA=CUDA,
      device=DEVICE,
      path=PATH)

Completed  in (8s),  phi_loss: -3.945,  theta_loss: 15424.491,  ess: 1.030
Completed  in (8s),  phi_loss: -18.779,  theta_loss: 4542.320,  ess: 1.059
Completed  in (8s),  phi_loss: -32.533,  theta_loss: 4266.087,  ess: 1.059
Completed  in (8s),  phi_loss: -37.118,  theta_loss: 4041.823,  ess: 1.061
Completed  in (8s),  phi_loss: -46.896,  theta_loss: 3806.028,  ess: 1.066
Completed  in (8s),  phi_loss: -47.991,  theta_loss: 3710.010,  ess: 1.064
Completed  in (8s),  phi_loss: -53.119,  theta_loss: 3519.406,  ess: 1.065
Completed  in (8s),  phi_loss: -55.561,  theta_loss: 3547.026,  ess: 1.072
Completed  in (8s),  phi_loss: -55.085,  theta_loss: 3457.389,  ess: 1.067
Completed  in (8s),  phi_loss: -58.928,  theta_loss: 3318.284,  ess: 1.070
Completed  in (8s),  phi_loss: -58.852,  theta_loss: 3278.996,  ess: 1.074
Completed  in (8s),  phi_loss: -62.035,  theta_loss: 3257.823,  ess: 1.082
Completed  in (8s),  phi_loss: -60.504,  theta_loss: 3189.499,  ess: 1.088
Completed  in (8s),  phi_

In [ ]:
from torch.nn.functional import conv2d, normalize, softmax
batch = data[:BATCH_SIZE]
B, T, FP, _ = batch.shape
batch_expand = batch.transpose(0,1).unsqueeze(1).repeat(1, SAMPLE_SIZE, 1, 1, 1).view(T, SAMPLE_SIZE*BATCH_SIZE, FP, FP)
kernels = mnist_mean.repeat(BATCH_SIZE*SAMPLE_SIZE,1,1).unsqueeze(1)
convolved = conv2d(batch_expand, kernels, groups=int(BATCH_SIZE*SAMPLE_SIZE))
convolved = convolved.view(T, SAMPLE_SIZE, BATCH_SIZE, 37, 37).view(T, SAMPLE_SIZE, BATCH_SIZE, 37*37).permute(1,2,0,-1)

In [ ]:
convolved[0,0,0,:].max()

In [ ]:
softmax(convolved, dim=-1).max()

In [ ]:
batch = data[:BATCH_SIZE].cuda().to(DEVICE)
mnist_mean = mnist_mean.cuda().to(DEVICE)
q, convolved = enc_coor(SAMPLE_SIZE, batch, mnist_mean.repeat(SAMPLE_SIZE, BATCH_SIZE, 1, 1))

In [ ]:
conv_img = convolved.cpu().view(10, SAMPLE_SIZE, BATCH_SIZE, 37,37).data.numpy()
original_img = batch.cpu().data.numpy()

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
fs = 15
num_cols = 10*2
num_rows = BATCH_SIZE
gs = gridspec.GridSpec(num_rows, num_cols)
gs.update(left=0.0 , bottom=0.0, right=1.0, top=1.0, wspace=0.05, hspace=0.05)
fig = plt.figure(figsize=(fs, fs * num_rows / num_cols))
for i in range(num_rows):
    for j in range(num_cols):
        ax = fig.add_subplot(gs[i, j])
        ax.set_xticks([])
        ax.set_yticks([])
        if j < 10:
            ax.imshow(original_img[i, j], cmap='gray', vmin=0.0, vmax=1.0)
        else:
            ax.imshow(conv_img[j-10, 0, i], cmap='gray')